In [1]:
import os
from pymongo import MongoClient

### Connection to MongoDB

In [ ]:
mongodb_uri = os.environ.get('MONGODB_URI')
database_name = os.environ.get('DATABASE_NAME')

client = MongoClient(mongodb_uri)
database = client[database_name]

#### Connecting to audio clips collection

In [ ]:
audio_clips = database['modified_audio_clips'] # last updated audio clips collection

#### Update video field in audio clips collection

In [ ]:
pipeline = [
    {
        '$lookup': {
            'from': 'modified_audio_descriptions_xx',  # last updated audio_descriptions collection
            'localField': 'audio_description', 
            'foreignField': '_id', 
            'as': 'ad_ac'
        }
    }, {
        '$unwind': {
            'path': '$ad_ac'
        }
    }, {
        '$addFields': {
            'video': '$ad_ac.video'
        }
    }, {
        '$project': {
            'ad_ac': 0
        }
    }, {
        '$out': 'modified_audio_clips_x' # new audio clips collection 
    }
]

audio_clips.aggregate(pipeline)